In [22]:
import numpy as np
import math
import datetime

lambdaa = 80
lambdas = 200
taus = 200
taua = 200

def frameDevFunction(i, j, v):
    frameCostValue = min(math.pow(((j-i)-v),2), taus)
    return frameCostValue

def frameAccelCost(h, i, j):
    frameaccelerationcost = min(math.pow(((j-i) - (h-i)),2), taua)
    return frameaccelerationcost

def arrayPath(frameCount, extraDiag, v):
    
    #v is targetted speed up; v<=extraDiag
    print "Loading Array"
    errors = np.loadtxt('array.txt')
    print "Array Loaded"
    #Add our speed up cost function
    for i in range(0, frameCount):
        for j in range(i+1, i+extraDiag+2):
            if(j<frameCount and i<frameCount):
                errors[i,j] += lambdas * frameDevFunction(i, j, v)
    print "Speed Up Cost Added"
    
    #Array is populated now assess costs
    costmatrix = np.zeros((frameCount, frameCount))
    #First Pass of Populate
    for i in range(1,frameCount):
        for j in range(i+1, i+extraDiag+2):
            n1 = []    
            #Populate a new list based on the for loop
            for k in range(i-1, i-(extraDiag+2),-1):
                #Populate our list
                n1.insert(k, (errors[k, i] + lambdaa*frameAccelCost(k, i, j)))
            if(i< frameCount and j<frameCount):
                errors[i, j]=np.min(n1)               
                #Is this next line correct or should we not build on the prior result? Pseudo-code is ambiguous          
                costmatrix[i, j] = i - ((extraDiag+1) - np.argmin(n1))   
    print "Cost Matrix Calculated"
                         
                         
    #Second Pass Trace Back Min Cost
    #Multi-dimensional argmin to set initial values of s and d (basically the min value in the last few rows we can select)
    #Grab last few entries of diagonals
    #Then find minimum, translate back to the indices in our costmatrix
    lastframevalues = []
    endrange = 3
    for i in range(1, endrange+1):
        lastframevalues.extend(np.diagonal(costmatrix, i, 0, 1)[-(endrange-(i-1)):])
    print lastframevalues
    print np.argmin(lastframevalues)
    s= 
    d= 
    p = []
    while(s>1):
        #Prepend to List and work our way back up through the cost matrix
        p.insert(0, s)
        b = costmatrix[s, d]
        d = s
        s = b
    np.savetxt('framelist.txt',p)
    return p

arrayPath(165, 3, 4)

Loading Array
Array Loaded
Speed Up Cost Added
Cost Matrix Calculated
[157.0, 158.0, 159.0, 157.0, 158.0, 157.0]
0


C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:64: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


[2.0,
 6.0,
 10.0,
 14.0,
 18.0,
 22.0,
 26.0,
 30.0,
 34.0,
 38.0,
 42.0,
 46.0,
 50.0,
 54.0,
 58.0,
 62.0,
 66.0,
 70.0,
 74.0,
 78.0,
 82.0,
 86.0,
 90.0,
 94.0,
 98.0,
 102.0,
 106.0,
 110.0,
 114.0,
 118.0,
 122.0,
 126.0,
 130.0,
 134.0,
 138.0,
 142.0,
 146.0,
 150]